In [2]:
import numpy as np

W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [3]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [4]:
class Embedding:
    """
    Embedding 클래스는 단어 임베딩을 위한 클래스입니다.
    Attributes:
        params (list): 가중치 행렬 W를 포함하는 리스트.
        grads (list): 가중치 행렬 W와 같은 형태의 0으로 초기화된 기울기 리스트.
        idx (None or array-like): 인덱스를 저장하는 변수.
    Methods:
        __init__(W):
            클래스의 인스턴스를 초기화합니다.
            Args:
                W (numpy.ndarray): 가중치 행렬.
        forward(idx):
            주어진 인덱스를 사용하여 가중치 행렬에서 임베딩 벡터를 추출합니다.
            Args:
                idx (array-like): 인덱스 배열.
            Returns:
                numpy.ndarray: 추출된 임베딩 벡터.
        backward(dout):
            역전파 단계에서 기울기를 계산합니다.
            Args:
                dout (numpy.ndarray): 출력 기울기.
            Returns:
                None
    """
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None
    
embed = Embedding(W)
out = embed.forward([0, 1, 3])


In [5]:
print(out)

[[ 0  1  2]
 [ 3  4  5]
 [ 9 10 11]]


In [6]:
backward = embed.backward(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))
print(backward)

None


In [3]:
import sys
sys.path.append('common')
import numpy as np
from common.layers import Embedding

W_in = np.arange(1,13).reshape(4, 3)
idx = np.array([2,0,3,0])
idx_shape = idx.shape
dh = np.array([[0,1,1], 
               [1,0,1], 
               [1,1,0], 
               [1,1,1]])
dh_shpae = dh.shape
Emb = Embedding(W_in)
print("h\n",Emb.forward(idx))

Emb.backward(dh)
print("dL/dh\n",Emb.grads[0])



h
 [[ 7  8  9]
 [ 1  2  3]
 [10 11 12]
 [ 1  2  3]]
dL/dh
 [[2 1 2]
 [0 0 0]
 [0 1 1]
 [1 1 0]]


In [8]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        print("h\n",h)
        print("target_W\n",target_W)
        dout = dout.reshape(dout.shape[0], 1)
        print("dout\n",dout)
        dtarget_W = dout * h
        print("dtarget_W\n",dtarget_W)
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        print("dh\n",dh)
        return dh
    
W_out = np.arange(1, 13).reshape(3, 4)
print("W_out\n",W_out)
h = np.array([[0, -1, 1], 
              [-1, 0, 1], 
              [-1, -1, 1]])
t= np.array([2, 0, 0])
ds = np.array([1, 2, 3])

ED = EmbeddingDot(W_out.T)
print("------")
print("ED.forward\n",ED.forward(h, t))
print("------")
print("ED.backward\n",ED.backward(ds))
print("------")
print("ED.grads\n",ED.grads[0])

W_out
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
------
ED.forward
 [4 8 3]
------
h
 [[ 0 -1  1]
 [-1  0  1]
 [-1 -1  1]]
target_W
 [[ 3  7 11]
 [ 1  5  9]
 [ 1  5  9]]
dout
 [[1]
 [2]
 [3]]
dtarget_W
 [[ 0 -1  1]
 [-2  0  2]
 [-3 -3  3]]
dh
 [[ 3  7 11]
 [ 2 10 18]
 [ 3 15 27]]
ED.backward
 [[ 3  7 11]
 [ 2 10 18]
 [ 3 15 27]]
------
ED.grads
 [[-5 -3  5]
 [ 0  0  0]
 [ 0 -1  1]
 [ 0  0  0]]
